In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

nlp = spacy.load('en_core_web_lg')


2023-03-08 22:02:12.666971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 22:02:12.666989: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/marko/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-08 22:02:14.395266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 22:02:14.395296: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 22:

In [3]:
df = pd.read_pickle("./pickle_files/df_odvojeni_word_vectors.pkl")

In [4]:
#spajanje odvojenih postova

df = df.groupby(['type', 'avgWordLen'], sort=False,as_index=False).agg({'posts':' '.join,'upperCount':'first','stopWordCount':'first','urlCount':'first','wordCount':'first','avgWordLen':'first'})

### nospacy ###

In [5]:
sequence_length = 1703
max_features = 2000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features, oov_token=oov_token)
tokenizer.fit_on_texts(df['posts'].array)
word_index= tokenizer.word_index
vectorList = tokenizer.texts_to_sequences(list(df['posts']))
vectorList = tf.keras.utils.pad_sequences(vectorList, padding=pad_type, truncating=trunc_type, maxlen=sequence_length,dtype='int32')

### spacy ###

In [6]:
# #padding vektora
# def paddingEmbedded(a):
#   b = np.zeros([len(a), len(max(a, key=lambda x: len(x))), 300])
#   for i, j in enumerate(a):
#     if (len(j) == 0):
#       continue
#     b[i][0:len(j)] = j
#     b[i][len(j):] = np.zeros(300,)
#   return b


# df['vectors'] = df['vectors'].apply(
#     lambda x: np.array(list(filter(lambda y: y.ndim > 1, x))))
# df = df[df.vectors.map(lambda d: len(d)) >
#         0].reset_index().drop('index', axis=1)
# df['vectors'] = df['vectors'].apply(lambda x: np.array(
#     list(map(lambda y: y.reshape(300,), np.concatenate(x)))))
# vectorList = paddingEmbedded(df['vectors'].array)


In [7]:
training_size = 0.8

sentences = []
labels = []

for ind in df.index:
    sentences.append(vectorList[ind])
    labels.append(df['type'][ind])


In [8]:
from sklearn.model_selection import train_test_split
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(sentences, labels, test_size=1-training_size, random_state=42, stratify=labels)

In [9]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)


In [10]:
encoder = OneHotEncoder()
training_labels = encoder.fit_transform(training_labels.reshape(-1,1)).toarray()
testing_labels = encoder.fit_transform(testing_labels.reshape(-1,1)).toarray()

In [11]:
testing_sentences = np.array(list(testing_sentences))
training_sentences = np.array(list(training_sentences))
# testing_sentences = tf.convert_to_tensor(testing_sentences)
# training_sentences = tf.convert_to_tensor(training_sentences)


In [12]:
NEpoch = 32
batch_size = 32

In [13]:
model = tf.keras.models.load_model('saved_models/fullSet_nospacy')

2023-03-08 22:02:27.085407: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
predict = model.predict(testing_sentences)

55/55 [==============================] - 6s 104ms/step


In [15]:
import sklearn.metrics as skm

pred_max = np.argmax(predict,axis=1)
test_max = np.argmax(testing_labels,axis=1)

print(skm.classification_report(test_max,pred_max))

              precision    recall  f1-score   support

           0       0.43      0.58      0.50        90
           1       0.63      0.42      0.50       149
           2       0.76      0.88      0.82       834
           3       0.83      0.70      0.76       662

    accuracy                           0.75      1735
   macro avg       0.66      0.64      0.64      1735
weighted avg       0.76      0.75      0.75      1735



In [20]:
sentences[0]

array([107,  55, 272, ...,   0,   0,   0], dtype=int32)